# DBT Lab
In this lab, we're going to combine the last tool we explored, Snowflake, and use it via DBT so we can see how together they make a more complete data analysis system.  There are two ways to run DBT, one similar to Snowflake where we can go via a Web UI, but to really leverage the open source and the capabilities of team collaboration, we'll use the command line version.  We'll set up from scratch and tie into our Snowflake account.

 - 1 Install dbt at the command line
 - 2 Set up config file to connect and test connection to Snowflake
 - 3 Query our data using SQL
 
 

## 1 Installing DBT at the command line
In this step, we'll create a quick virtual environment and install dbt, then setup our first project.  We'll use `first-project` as the name of our first project in the following steps:

### Requirements
* Python 3.6 or higher


* Create a virtual environment with `python3 -m venv env`
* Activate the environment with `source env/bin/activate`
* Install with `pip install dbt`
* Initialize a project with `dbt init first-project`

* NOTE: If you see with markupsafe, use `pip uninstall markupsafe` and re-install `pip install markupsafe==2.0.1`

## 2 Set up connection to and test Snowflake
Let's now set up the yaml config file needed to connect DBT with Snowflake.

For reference: [Supported Data Platforms](https://docs.getdbt.com/docs/supported-data-platforms)

### Requirements
For this section, you will need three things from the snowflake account we set up in the previous lecture
* Username (EOlivaresEVI)
* Password (********)
* The first portion of the url which is your account id.  (rja95216)

### 2.1 Preparing Snowflake
Let's create a warehouse for use with our DBT connection, as well as a user specifically for this use.
* Log into your snowflake account.
* Using the role `ACCOUNTADMIN`, go to `Admin->Warehouses`
* Lets use...
    - name: dbt_warehouse
    - size: X-Xmall 1 credit/hour
    - Advanced Warehouse Options/Suspend After (min): 5
    
#### Add a specific dbt user and role
* Now let's go to `Admin/Users & Roles`
* Use the `[+User]` button to create a new user
* Select a name, and a password, and write them down, as we'll use them in the configuration step next.
* Uncheck the box for `Force user to change password on first time login`
* dbt_transform/DbtTra@nsform1
* Now let's got o `Roles` and `[+Role]`
* Note the name of the new role, then from the `Roles/Table` use the `[...]` button on the right to select `Grant/Grant to User`
* Select your user and then repeat the process to grant to the user specific to dbt. (We want both, one to use with our connection, and one to use if we need to debug)

#### Add a database and schema
* Go to `Data/Databases` and create `ANALYTICS` database
* After creating it, open the link to it on the UI and go to the `[+privileges]` button on the right
* Select the role `ACCOUNTADMIN` and grant privileges for `CREATE SCHEMA`, `MODIFY`, `USAGE`.
* Repeat the process for the `DBT_TRANSFORM_ROLE`
* Now let's go to a worksheet so we can create a schema in the database.
* In the worksheet, let's execute `CREATE SCHEMA analytics.dbt;`
* Click on the dabase view on the left menu, and refresh.  We should be able to see our new schema now.

### 2.2 Setting up the connection
When we ran `dbt init` it set up a config for us.  We can run `dbt debug --config-dir` to find it's default location. We'll need to edit this file to set up our Snowflake location.  Open the file, by default at `~/.dbt/profiles.yml` and it should look like this:
```yaml
jaffle_shop: # this needs to match the profile: in your dbt_project.yml file
  target: dev
  outputs:
    dev:
      type: bigquery
      method: service-account
      keyfile: /Users/eolivares/.dbt/dbt-practice-210412-b994f6a6e03a.json # replace this with the full path to your keyfile
      project: dbt-practice-210412 # Replace this with your project id
      dataset: dbt_efrain # Replace this with dbt_your_name, e.g. dbt_bob
      threads: 1
      timeout_seconds: 300
      location: US
      priority: interactive
```

We'll replace it wiht the following
```yaml
dbt-snowflake-connection: # this needs to match the profile: in your dbt_project.yml file
  target: dev
  outputs:
    dev:
      type: snowflake
      account: rja95216

      user: dbt_transform
      password: <the password set for dbt_transform>

      role: dbt_transform_role


      dataset: analytics
      warehouse: dbt_warehouse
      schema: dbt
      threads: 1                         # number of models to run at a time, we'll start with 1
      client_session_keep_alive: False   # do not keep snowflake session open after a run (save credits)
```

#### 2.3 Final configuration in dbt_project.yml
Now that we set up the connection yaml, we'll need to put everything together in our working dbt space at the cli.
Back at the directory where you executed `dbt init`
* Open `dbt_project.yml`
* Pick a name for the project for `name:`.  This will be used in the name of the path to store sql as we'll see next.  Say for example, `dbt_first_project`
* The `profile:` should be set to the same name as you used in `.dbt/profiles.yml`
* Near the bottom, under `models:`, use the name of the project so it looks like this
```yaml
...
models:
  dbt_first_project:
    # Config indicated by + and applies to all files under models/example/
    example:
      +materialized: view
```

Debugging tips:
* Some urls, where you get your account from for snowflake, include a region, so if you see something like `us-west-1` just before the `snowflake...` text in the url, it is part of your account name.
* The username is not case sensitive, thus `dbt_transformer` is the same as `DBT_TRANSFORMER`, however, as expected, the password is case sensitive.



# 3 
